# Initialization

In [1]:
import os
from pathlib import Path
import signal
import time

# PyBoolNet initialization
from pyboolnet.repository import get_all_names, get_bnet
from pyboolnet.file_exchange import bnet2primes
from pyboolnet.trap_spaces import compute_trap_spaces

# BioLQM init
import biolqm

# trappist init
import trappist

# timeout with signals
class TimeOutException(Exception):
   pass

def handler(_signnum, _frame):
    print("Timeout")
    raise TimeOutException()
    
signal.signal(signal.SIGALRM, handler)

ModuleNotFoundError: No module named 'biolqm'

In [2]:
# compute trapspaces using PyBoolNet directly
def bench_pyboolnet(models, max=1000, timeout=120):
    for model in models:
        print(model)
        primes = None
        signal.alarm(timeout)
        try:
            start = time.perf_counter()
            primes = bnet2primes(model + ".bnet")
            tspaces = compute_trap_spaces(primes, "min", max_output=max)
            end = time.perf_counter()
            print(f"{len(tspaces)} trap spaces {end - start:.2f}\n===")
        except TimeOutException:
            print(f"DNF{'(bnet2primes)' if primes is None else ''}\n===")
        signal.alarm(0)

In [3]:
# compute trapspaces using trappist
def bench_trappist(models, max=1000, timeout=120, method="asp"):
    for model in models:
        print(model)
        signal.alarm(timeout)
        try:
            start = time.perf_counter()
            biolqm.save(biolqm.load(model + ".bnet"), model + ".pnml")
            # end = time.perf_counter()
            # print(f"conversion: {end - start}s")
            tspaces = trappist.compute_trap_spaces(model + ".pnml", max_output=max, method=method)
            end = time.perf_counter()
            print(f"{len(list(tspaces))} trap spaces {end - start:.2f}\n===")
        # we need to catch our TimeOutException, but it raises a Py4J exception too…
        except:
            print("DNF\n===")
        signal.alarm(0)

## PyBoolNet repository

Unfortunately all models are very easy

In [4]:
# all models from the PyBoolNet repository https://github.com/hklarner/pyboolnet/tree/master/pyboolnet/repository
models = get_all_names()

# only the slowest ones
# models = ['jaoude_thdiff', 'n12c5', 'remy_tumorigenesis', 'selvaggio_emt']

# save .bnet files locally
for model in models:
    net = get_bnet(model)
    with open(model + ".bnet", "wt") as f:
        print(net, file=f)

In [ ]:
bench_pyboolnet(models)

In [ ]:
bench_trappist(models, method="asp")

In [ ]:
bench_trappist(models, method="sat")

## Local set of big models

Collected from various bibliographic sources (see article for details)

In [7]:
# all models from the trappist's benchmark directory
models = [str(Path().absolute() / "models" / name[:-5]) for name in os.listdir("models") if name.endswith(".bnet")]

In [ ]:
bench_pyboolnet(models)

In [ ]:
bench_trappist(models, method="asp")

In [ ]:
bench_trappist(models, max=0, method="asp")

In [ ]:
bench_trappist(models, method="sat")

In [ ]:
bench_trappist(models, max=0, method="sat")